<a href="https://colab.research.google.com/github/olvad/ML_course_openEdu/blob/main/Homeworks/HW_5_Decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# скачиваем файл
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl' -O heart.csv

--2024-07-16 18:31:48--  https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl
Resolving docs.google.com (docs.google.com)... 173.194.174.101, 173.194.174.102, 173.194.174.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.174.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl&export=download [following]
--2024-07-16 18:31:48--  https://drive.usercontent.google.com/download?id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.23.132, 2404:6800:4008:c02::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.23.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11328 (11K) [application/octet-stream]
Saving to: ‘heart.csv’

heart.csv           100%[===================>]  11.06K  --.-KB/s    in 0s  

In [ ]:
# устанавливаем нужную версию библиотеки sklearn, чтобы ответы сошлись
!pip install scikit-learn==0.22.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
Failed to build scikit-learn
ERROR: Could not build wheels for scikit-learn, which is required to install pyproject.toml-based projects


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

В этом задании вы будете работать с [данными о пациентах, у части которых есть заболевание сердца](https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data). Вам нужно построить модель, классифицирующую пациентов на больных этим заболеванием и тех, у кого его нет, а также проанализировать результаты.

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

1. Обучите решающее дерево из `sklearn` (`sklearn.tree.DecisionTreeClassifier`) без ограничения на максимальную глубину (`max_depth=None`). В качестве сида поставьте `random_state=13`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

   Найдите долю правильных ответов полученного алгоритма на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [ ]:
from sklearn.tree import DecisionTreeClassifier


dtc = DecisionTreeClassifier(random_state=13)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=13)

In [ ]:
from sklearn.metrics import accuracy_score

round(accuracy_score(dtc.predict(X_train), y_train) * 100,2)

100.0

2. Теперь найдите долю правильных ответов полученного алгоритма на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Заметно ли переобучение?

In [ ]:
round(accuracy_score(dtc.predict(X_test), y_test) * 100,2)

67.11

3. Подберите с помощью кросс-валидации оптимальные гиперпараметры алгоритма. Выбирайте из следующих наборов:


- `max_depth`: [3, 4, 5, 6, 7, 8, 9, 10, None]
- `max_features`: ['auto', 'log2', None]
- `min_samples_leaf`: range(1, 10)
- `min_samples_split`: range(2, 10)
- `criterion`: ['gini', 'entropy']

    В этом вам поможет метод `sklearn.model_selection.GridSearchCV`. Подробнее о том, какие параметры и методы в нем используются, и о примерах работы с ним можно прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
    
    1) Создайте решающее дерево - не забудьте поставить `random_state=13`.
    
    2) Задайте `param_grid` - сетку (словарь) гиперпараметров и их значений, по которой будет проходить метод.
    
    3) Вызовите метод `GridSearchCV` - в качестве параметра `estimator` задайте решающее дерево из первого шага, `param_grid` - сетку из второго. Задайте параметр `cv=5`, чтобы кросс-валидация проходила по 5 фолдам. Также задайте параметр `scoring='accuracy'`, чтобы оценка качества моделей на кросс-валидации проходила с помощью доли правильных ответов. Запустите метод на обучающей выборке с помощью `fit`.
    
    4) Выведите лучшие параметры с помощью атрибута `best_params_`.
    
    Какое значение глубины дерева получилось оптимальным?

In [ ]:
params = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
    'max_features': ['auto', 'log2', None],
    'min_samples_leaf': range(1, 10),
    'min_samples_split': range(2, 10),
    'criterion': ['gini', 'entropy']
}



In [ ]:
from sklearn.model_selection import GridSearchCV

dtc = DecisionTreeClassifier(random_state=13)
gscv = GridSearchCV(estimator = dtc, param_grid = params, cv = 5,
                    scoring = 'accuracy')


In [ ]:
gscv.fit(X_train, y_train)

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explici

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=13),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
                         'max_features': ['auto', 'log2', None],
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10)},
             scoring='accuracy')

In [ ]:
params = gscv.best_params_

4. Какое лучшее усредненное значение доли правильных ответов получилось на кросс-валидации (для оптимальных значений гиперпараметров)? Вам поможет атрибут `best_score_`. Ответ округлите до двух знаков после запятой и дайте в процентах.

In [ ]:
round(gscv.best_score_ * 100, 2)

83.24

5. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

In [ ]:
round(accuracy_score(gscv.predict(X_train), y_train) * 100, 2)

90.75

In [ ]:
accuracy_score(gscv.best_estimator_.predict(X_train), y_train)

0.9074889867841409

6. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Уменьшилось ли переобучение?

In [ ]:
round(accuracy_score(gscv.predict(X_test), y_test) * 100, 2)

73.68

7. Решающее дерево позволяет предсказывать не только классы, но и вероятности классов - с помощью метода `predict_proba`. Посмотрите на вероятности классов полученного решающего дерева и посчитайте значение AUC-ROC. Ответ округлите до двух знаков после запятой.

In [ ]:
from sklearn.metrics import roc_auc_score

predd = gscv.predict_proba(X_test)[:, 1]

auc_roc = roc_auc_score(y_test, predd)

In [ ]:
round(auc_roc, 2)

0.77

8. Какой признак является самым важным по мнению полученного решающего дерева? Чтобы это проверить, вам поможет атрибут `feature_importances_`.

In [ ]:

pd.DataFrame({
    'znach': gscv.best_estimator_.feature_importances_,
    'column': X_train.columns
}
)


,znach,column
0,0.007687,age
1,0.118580,sex
2,0.000000,cp
3,0.006221,trestbps
4,0.030316,chol
5,0.000000,fbs
6,0.005542,restecg
7,0.155527,thalach
8,0.153834,exang
9,0.065503,oldpeak
